# COMP9318 Lab2

## Instructions
1. This note book contains instructions for **COMP9318-Lab2**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are only allowed to use **Python 3.6** for implementation.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures return by corresponding functions.

* You need to submit the code for **Lab2** via following link: https://kg.cse.unsw.edu.au/submit/

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* If you choose to skip a question, leave the corresponding function body as it is (i.e., keep the `pass` line), otherwise it may affect your mark for other questions.

* You are allowed to add other functions and/or import additional modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **20:59:59 on 23rd March, 2020 (Sydney Time)**. We will **not** accept any late submissions.

# Question 1: Optimized BUC algorithm (100 points)

You need to implement the full `buc_rec_optimized` algorithm with the single-tuple optimization (as described below). Given an input dataframe:

 A | B | M 
---|---|---
 1 | 2 | 100
 2 | 1 | 20

Invoking  `buc_rec_optimized` on this data will result in following dataframe: 


 A | B | M
---|---|---
 1 | 2 | 100
 1 |ALL| 100
 2 | 1 | 20 
 2 |ALL| 20
ALL| 1 | 20
ALL| 2 | 100
ALL|ALL| 120

We have pre-defined the function `buc_rec_optimized` in the file `submission.py`, and its helper functions are defined in the file `helper.py`. 

**Note:** You should use the functions defined in the file `helper.py`, you are not allowed to change this file. We will provide this file in the test environment.

## Input and output

Both `input` and `output` are dataframes.

The input dataframe (i.e., the base cuboid) is directly generated from the input file. Given the dimensionality of the base cuboid is $d$, each row is like:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 

The output dataframe contains $n$ rows, each for a non-empty cell in the compute data cube derived from the input base cuboid. Each row is formatted like input:

<pre>
v_1  v_2 ...  v_d  m
</pre>

where v_i is the cell's value on the i-th dimension, and m is the measure value. 


## The single-tuple optimization

Consider the naive way of recursive implementation of the BUC algorithm, you will notice that it uses several recursive calls to compute all the derived results from an input that consists of only one tuple. This is certainly a waste of computation. 

For example, if we are asked to compute the cube given the following input

 B | C | M 
---|---|---
 1 | 2 | 100

We can immmediately output the following, **without** using any recursive calls. 

<pre>
1    2    100
*    2    100
1    *    100
*    *    100
</pre>



** Note: For lab-2, you are allowed to use only two libraries, i.e., pandas, and numpy.** 

In [1]:
import pandas as pd
import numpy as np

In [2]:
##============================================================
# Data file format: 
# * tab-delimited input file
# * 1st line: dimension names and the last dimension is assumed to be the measure
# * rest of the lines: data values.

def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)
      
# helper functions
def project_data(df, d):
    # Return only the d-th column of INPUT
    return df.iloc[:, d]

def select_data(df, d, val):
    # SELECT * FROM INPUT WHERE input.d = val
    col_name = df.columns[d]
    return df[df[col_name] == val]

def remove_first_dim(df):
    # Remove the first dim of the input
    return df.iloc[:, 1:]

def slice_data_dim0(df, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(df, 0, v)
    return remove_first_dim(df_temp)

In [3]:
def buc_rec_optimized(df):  # do not change the heading of the function

    if df.shape[0] == 1:
        df_out = one_dim_duc(df)
    else:
        header = list(df)
        df_out = pd.DataFrame(columns=header)
        _buc_rec_optimized(df, [], df_out)
    return df_out


def deepcopy(list):
    b = []
    for i in list:
        b.append(i)
    return b


def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    return (df)


def output(val):
    print('=>\t{}'.format(val))


def one_dim_duc(df):
    vals = list(df.loc[0])
    result = [vals[:-1]]
    for i, val in enumerate(result):
        temp = deepcopy(val)
        for j, list_val in enumerate(temp):
            if list_val != 'ALL':
                temp2 = deepcopy(temp)
                temp2[j] = 'ALL'
                if temp2 not in result:
                    result.append(temp2)

    for i in result:
        i.append(vals[-1])
    result = pd.DataFrame(result, columns=list(df))
    return result


def _buc_rec_optimized(df, pre_num, df_out):  # help function
    # Note that input is a DataFrame
    dims = df.shape[1]

    if dims == 1:
        # only the measure dim
        input_sum = sum(project_data(df, 0))
        pre_num.append(input_sum)

        df_out.loc[len(df_out)] = pre_num

    else:
        # the general case

        dim0_vals = set(project_data(df, 0).values)
        print("dim0_vals" , dim0_vals)
        temp_pre_num = deepcopy(pre_num)
        print("temp_pre_num" , temp_pre_num)
        for dim0_v in dim0_vals:
            pre_num = deepcopy(temp_pre_num)
            print("pre_num" , pre_num)
            sub_data = slice_data_dim0(df, dim0_v)
            print("sub_data" , sub_data)
            pre_num.append(dim0_v)
            print("dim0_v" , dim0_v)
            print("pre_num" , pre_num)

            _buc_rec_optimized(sub_data, pre_num, df_out)
        ## for R_{ALL}
        sub_data = remove_first_dim(df)
        print("sub_data" , sub_data)

        pre_num = deepcopy(temp_pre_num)
        print("pre_num" , pre_num)
        pre_num.append("ALL")
        _buc_rec_optimized(sub_data, pre_num, df_out)

In [4]:
## You can test your implementation using the following code...
import submission as submission
input_data = read_data('./asset/a_.txt')
output = buc_rec_optimized(input_data)
output

AttributeError: module 'helper' has no attribute 'read_data'